In [8]:
! pip install py7zr pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.0 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [py7zr]32m8/9 [py7zr]todomex]


In [ ]:
import requests
import pandas as pd
import numpy as np
import gzip
import py7zr
import io

In [ ]:
#Fonction pour récupérer les données d'un fichier csv.gz ( pour les données météorologiques )
def recuperer_donnees_gz(dataset_id, fichier_csv, sep):
    """
    Cette fonction télécharge les données d'un fichier CSV.GZ à partir d'une URL et les charge dans un DataFrame pandas.
    
    Paramètres:
    dataset_id (str): identifiant du fichier CSV.GZ.
    fichier_csv (str): Le nom du fichier CSV.GZ à enregistrer localement.
    sep ( str ) : le séparateur à utiliser
    
    Retourne:
    DataFrame: Un DataFrame pandas contenant les données du fichier CSV.
    """

    # URL de base pour accéder à l'API
    base_api = "https://www.data.gouv.fr/api/1/"

    # Chemin pour accéder aux enregistrements du dataset
    key_api = "datasets/r/"

    # Construction de l'URL complète
    url = f"{base_api}{key_api}{dataset_id}"


    response = requests.get(url)
    response.raise_for_status()
    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as gz:
        df = pd.read_csv(gz, sep = sep)
    
    return df

In [6]:
quot_25 = recuperer_donnees_gz('92065ec0-ea6f-4f5e-8827-4344179c0a7f', 'QUOT_2020-2025', ';')
quot_25.tail()

,LAMBX,LAMBY,DATE,PRENEI,PRELIQ,T,FF,Q,DLI,SSI,...,RESR_NEIGE6,HTEURNEIGE,HTEURNEIGE6,HTEURNEIGEX,SNOW_FRAC,ECOULEMENT,WG_RACINE,WGI_RACINE,TINF_H,TSUP_H
21079847,11960,17450,20251027,0.0,0.0,15.5,2.4,6.076,2695.1,1376.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.205,0.0,12.1,20.3
21079848,11960,17450,20251028,0.0,0.1,15.4,2.1,7.500,2877.0,1195.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.205,0.0,11.2,22.5
21079849,11960,17450,20251029,0.0,1.0,17.0,2.9,9.242,3156.7,641.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.205,0.0,12.0,22.7
21079850,11960,17450,20251030,0.0,3.3,17.7,1.6,11.344,3426.6,178.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.205,0.0,16.5,18.9
21079851,11960,17450,20251031,0.0,0.4,17.9,1.6,11.022,3174.7,859.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.207,0.0,16.0,21.5


Il faudra repliquer la même chose sur toutes les bases depuis 1958, et fusionner toutes les bases en une seule complète de 1958 à 2025

In [ ]:
# Base des swi uniformes

def recuperer_donnees_7z(dataset_id, fichier_csv, sep):
    """
    Cette fonction télécharge les données d'un fichier CSV.GZ à partir d'une URL et les charge dans un DataFrame pandas.
    
    Paramètres:
    dataset_id (str): identifiant du fichier CSV.GZ.
    fichier_csv (str): Le nom du fichier CSV.GZ à enregistrer localement.
    sep ( str ) : le séparateur à utiliser
    
    Retourne:
    DataFrame: Un DataFrame pandas contenant les données du fichier CSV.
    """

    # URL de base pour accéder à l'API
    base_api = "https://www.data.gouv.fr/api/1/"

    # Chemin pour accéder aux enregistrements du dataset
    key_api = "datasets/r/"

    # Construction de l'URL complète
    url = f"{base_api}{key_api}{dataset_id}"


    response = requests.get(url)
    response.raise_for_status()
    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as gz:
        df = pd.read_csv(gz, sep = sep)
    
    return df